# MCJ-CloudHub試用環境作成

---

MCJ-CloudHub動作確認（試用）用環境作成Notebook。  
MoodleとMCJ-CloudHubを１つのVM上で構築する。  

## 事前準備
- VCノード構築可能なVM * 1
- `/notebooks/notebook/token.txt` にテキストファイルとしてVCCアクセストークンが保存されている
- `/notebooks/notebook/certs/fullchain.pem` としてSSL証明書が保存されている
- `/notebooks/notebook/certs/privkey.pem` としてSSL証明書の秘密鍵が保存されている

## 流れ
1. VCノード作成
1. MCJ-CloudHub構築
1. Moodle構築
1. Moodle外部ツール設定
1. MCJ-CloudHub設定ファイル更新（Moodleで外部ツール設定後に生成されるシークレット情報を設定する）

## はじめに

このNotebookではJupyterHub環境を構築するためのノード作成を行います。managerノードにNFSサーバを配置する構成とします。  
本ノートブックで構築する部分を以下に図示します。

![モジュール構成](images/011/arch-011.png)

In [ ]:
# ユーザごとに異なる項目

ugroup_name = 
vc_ipaddress = 
worker_ipaddresses = [
]

# JupyterHubのホスト名(FQDN) (例) 'www.sample.org'
jupyterhub_fqdn = 

# moodle
import random
import string
moodle_db_password = ''.join([random.choice("abcdefghijklmnopqrstuvwxyz" + string.digits) for _ in range(32)])
moodle_admin_password = ''.join([random.choice("abcdefghijklmnopqrstuvwxyz" + string.digits) for _ in range(10)])

print(f'moodle_db_password={moodle_db_password}')
print(f'moodle_admin_password={moodle_admin_password}')

## VCノード作成・Moodle起動  

以下まとめ実行する。

### VCノード作成

In [ ]:
import os
from getpass import getpass

with open('/notebooks/notebook/token.txt', 'r') as f:
    vcc_access_token = f.read()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

In [ ]:
vc_provider = 'onpremises'
manager_flavor = 'default'
manager_disk_size = 16
worker_flavor = 'default'
worker_disk_size = 16
worker_nodes = 0
ssh_public_key_path = '~/.ssh/id_rsa.pub'
ssh_private_key_path = '~/.ssh/id_rsa'
ssh_user = 'mdxuser'
docker_address_pool = '10.10.0.0/16'

In [ ]:
%run scripts/check_params.py
check_parameters(
    'ugroup_name',
    'vc_provider',
    'manager_flavor',
    'manager_disk_size',
    'worker_flavor',
    'worker_disk_size',
    'worker_nodes',
    'vc_ipaddress',
    'worker_ipaddresses',
    'vc_mac_address',
    'worker_mac_addresses',
    'docker_address_pool',
    'ssh_public_key_path',
    'ssh_private_key_path',
    params={
        'vcp': vcp,
        'opt_vars': [
            'vc_ipaddress', 'worker_ipaddresses',
            'vc_mac_address', 'worker_mac_addresses',
        ],
    },
    nb_vars=locals(),
)

if vc_provider == 'onpremises':
    if not ssh_user:
        raise NotImplementedError
        print('ssh_user is required if construct on onpremises')
    
    worker_flavor = 'default'
    manager_flavor = 'default'

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
import os
%run scripts/group.py
update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
    vc_provider=vc_provider,
    manager_flavor=manager_flavor,
    manager_disk_size=manager_disk_size,
    worker_flavor=worker_flavor,
    worker_disk_size=worker_disk_size,
    worker_nodes=worker_nodes,
    docker_address_pool=docker_address_pool,
    ssh_public_key_path=os.path.expanduser(ssh_public_key_path),
    ssh_private_key_path=os.path.expanduser(ssh_private_key_path),
    moodle_db_password=moodle_db_password,
    moodle_admin_password=moodle_admin_password,
    jupyterhub_fqdn=jupyterhub_fqdn,
)
if 'vc_ipaddress' in vars():
    update_group_vars(ugroup_name, vc_ipaddress=vc_ipaddress)
if 'worker_ipaddresses' in vars():
    update_group_vars(ugroup_name, worker_ipaddresses=worker_ipaddresses)
if 'vc_mac_address' in vars():
    update_group_vars(ugroup_name, vc_mac_address=vc_mac_address)
if 'worker_mac_addresses' in vars():
    update_group_vars(ugroup_name, worker_mac_addresses=worker_mac_addresses)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}

In [ ]:
# (例)
# vc_nfs_disk_size = 32

vc_nfs_disk_size = 0

VCディスクのためのUnitGroup名を指定してください。ここではVCノードのUnitGroup名から自動的に導き出した名前を設定します。

In [ ]:
disk_unit_group = ugroup_name + '_disk'
print(disk_unit_group)

この章で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py

if 'vc_nfs_disk_size' in vars() and vc_nfs_disk_size > 0:
    if vc_provider in ['aws', 'azure']:
        update_group_vars(
            ugroup_name,
            disk_unit_group=disk_unit_group,
            vc_nfs_disk_size=vc_nfs_disk_size,
        )
    else:
        del(vc_nfs_disk_size)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}

UnitGroupを作成します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

if 'disk_unit_group' in gvars and 'vc_nfs_disk_size' in gvars and gvars['vc_nfs_disk_size'] > 0:
    ug_disk = vcp.create_ugroup(gvars['disk_unit_group'], ugroup_type='storage')

UnitGroup作成後の一覧を表示させます。

In [ ]:
vcp.df_ugroups()

### VCディスクの作成

NFS用のVCディスクを作成します。

In [ ]:
if 'ug_disk' in vars() and gvars['vc_nfs_disk_size'] > 0:
    nfs_disk_spec = vcp.get_spec(gvars['vc_provider'] + '_disk', 'small')
    if gvars['vc_provider'] == 'azure':
        nfs_disk_spec.disk_size_gb = gvars['vc_nfs_disk_size']
    elif gvars['vc_provider'] == 'oracle':
        nfs_disk_spec.size_in_gbs = gvars['vc_nfs_disk_size']
    else:
        nfs_disk_spec.size = gvars['vc_nfs_disk_size']
    ug_disk.create_unit('nfs', nfs_disk_spec)

作成したVCディスクの一覧を表示します。

In [ ]:
from IPython.display import display
if 'ug_disk' in vars():
    display(ug_disk.df_nodes())

### managerノード

#### manager用のVCノードを起動する

manager用VCノードの `spec` を指定します。

In [ ]:
import sys
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
spec_mgr = vcp.get_spec(gvars['vc_provider'], gvars['manager_flavor'])

# Baseコンテナイメージを指定する
spec_mgr.image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:base-nfsd'
spec_mgr.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

if 'vc_ipaddress' in gvars:
    # manager用VCノードに割り当てるIPアドレスを指定する
    spec_mgr.ip_addresses = [gvars['vc_ipaddress']]
elif 'vc_mac_addresses' in gvars:
    # manager用VCノードに割り当てるMACアドレスを指定する
    spec_mgr.mac_addresses = [gvars['vc_mac_address']]
    
# ルートボリュームサイズを指定する
if vc_provider == 'aws':
    spec_mgr.volume_size = gvars['manager_disk_size']
elif vc_provider == 'azure':
    spec_mgr.disk_size_gb = gvars['manager_disk_size']
else:
    print('This provider does not support the specification of the root volume size.' +
          ' Does not set the disk size.', file=sys.stderr)

# オンプレミスに構築する場合、SSHログインユーザ名を指定
if vc_provider == 'onpremises':
    spec_mgr.user_name = ssh_user

# VCノードにsshでログインするための公開鍵を指定する
spec_mgr.set_ssh_pubkey(gvars['ssh_public_key_path'])
spec_mgr.params_e.append(f"UGROUP_NAME={ugroup_name}")

if 'ug_disk' in vars():
    spec_mgr.disks = ug_disk.find_nodes()
    if len(spec_mgr.disks) > 0:
        spec_mgr.params_e.append("NFS_MKFS=yes")
else:
    spec_mgr.params_v.append('/exported:/exported')

`spec` の設定値を確認します。

In [ ]:
print(spec_mgr)

UnitGroupを作成します。

In [ ]:
ugroup = vcp.create_ugroup(ugroup_name)

manager用VCノードを起動します。

In [ ]:
unit_mgr = ugroup.create_unit('manager', spec_mgr)

起動したVCノードの一覧を表示します。

vcname:　本ノートブックで指定した、`UnitGroup`  
unit_name:　`manager`

In [ ]:
unit_mgr.df_nodes()

VCノードのIPアドレスを変数`vc_ipaddress`に設定します。

In [ ]:
vc_ipaddress = unit_mgr.find_ip_addresses()[0]
print(vc_ipaddress)

`group_vars`ファイルにIPアドレスの値を記録します。

In [ ]:
update_group_vars(ugroup_name, vc_ipaddress=vc_ipaddress)
gvars = load_group_vars(ugroup_name)

#### managerノードに対するAnsibleの設定

起動したVCノードをAnsibleで操作するための設定を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in unit_mgr.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
%run scripts/group.py
%run scripts/edit_conf.py

inventory = {'all': {'children': {
    ugroup.name: {
        'children': {
            f'{ugroup.name}_{unit_mgr.name}': {
                'hosts': dict([(x, {}) for x in unit_mgr.find_ip_addresses()]),
            },
        },
        'vars': {
            'ansible_user': 'vcp',
            'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
            'ansible_python_interpreter': '/usr/bin/python3',
        },
    },
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

 先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。
> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
cfg = setup_ansible_cfg()
generate_edit_link(cfg)

次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。コンフィギュレーションファイルの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat ansible.cfg

UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup.name} -m ping

#### NFSサーバの確認

managerノードのNFSサーバのサービス状態を確認します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -a 'systemctl status nfs-server'

NFSエクスポートの状態を確認します。managerノードの起動時は全てのノードに対してアクセス可能な状態に設定されています。後ほどworkerノードを起動してNFSクライアントのIPアドレスが確定した時点で`/etc/exports`の設定を更新し、アクセスできるノードの制限を行います。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -a 'exportfs -v'

ディレクトリのパーミッションを設定します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -m file -a 'path=/home mode=777'
!ansible {ugroup.name}_{unit_mgr.name} -b -m file -a 'path=/exchange mode=777'

### workerノード

#### worker用のVCノードを起動する

worker用VCノードの `spec` を指定します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
spec_worker = vcp.get_spec(gvars['vc_provider'], gvars['worker_flavor'])

# Baseコンテナイメージを指定する
spec_worker.image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:base'

spec_worker.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

# workerノード数を指定する
spec_worker.num_nodes = int(gvars['worker_nodes'])

if 'worker_ipaddresses' in gvars:
    # worker用VCノードに割り当てるIPアドレスを指定する
    spec_worker.ip_addresses = gvars['worker_ipaddresses']
elif 'vc_mac_addresses' in gvars:
    # worker用VCノードに割り当てるMACアドレスを指定する
    spec_worker.mac_addresses = gvars['worker_mac_addresses']

# ルートボリュームサイズを指定する
if vc_provider == 'aws':
    spec_worker.volume_size = gvars['worker_disk_size']
elif vc_provider == 'azure':
    spec_worker.disk_size_gb = gvars['worker_disk_size']
else:
    print('This provider does not support the specification of the root volume size.' +
          ' Does not set the disk size.', file=sys.stderr)

# オンプレミスに構築する場合、SSHログインユーザ名を指定
if vc_provider == 'onpremises':
    spec_worker.user_name = ssh_user

# VCノードにsshでログインするための公開鍵を指定する
spec_worker.set_ssh_pubkey(gvars['ssh_public_key_path'])

# NFSサーバのIPアドレスを指定する
spec_worker.params_e.append("NFS_SERVER=" + gvars['vc_ipaddress'])

`spec` の設定値を確認します。

In [ ]:
print(spec_worker)

worker用VCノードを起動します。

In [ ]:
unit_worker = ugroup.create_unit('worker', spec_worker)

起動したVCノードの一覧を表示します。

vcname:　本ノートブックで指定した、`UnitGroup`  
unit_name:　`worker`

In [ ]:
unit_worker.df_nodes()

#### workerノードに対するAnsibleの設定

VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

In [ ]:
!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in unit_worker.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
inventory = {'all': {'children': {
    ugroup.name: {
        'children': {
            f'{ugroup.name}_{unit_worker.name}': {
                'hosts': dict([(x, {}) for x in unit_worker.find_ip_addresses()]),
            },
        },
    },
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup.name} -m ping

#### NFSの設定

workerノードにおけるNFSマウントの状態を確認します。`/jupyter`, `/exchange`のエントリが存在していることを確認してください。

In [ ]:
!ansible {ugroup.name}_{unit_worker.name} -a 'mount -t nfs4'

NFSサーバの `/etc/exports` の設定を更新します。NFSサーバにアクセスできるNFSクライアントをworkerノードのみとなるように設定します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

exports_opts = 'rw,fsid=0,no_root_squash,no_subtree_check,sync,crossmnt'
with TemporaryDirectory() as workdir:
    exports = Path(workdir) / f'{ugroup_name}.exports'
    with exports.open(mode='w') as f:
        for addr in unit_worker.find_ip_addresses():
            f.write(f'/exported/{ugroup_name} {addr}({exports_opts})\n')
    !cat {exports}
    !ansible {ugroup.name}_{unit_mgr.name} -b -m copy -a \
        'src={exports} dest=/etc/exports.d/ backup=yes'

設定ファイルを読み込ませてエクスポート設定を更新します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -a 'exportfs -r -v'

NFSサーバ側に作成したファイルをNFSクライアント側で参照できることを確認します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -m file -a 'path=/jupyter/xxx state=touch'
!ansible {ugroup.name}_{unit_mgr.name} -m file -a 'path=/exchange/xxx state=touch'
!ansible {ugroup.name}_{unit_worker.name} -a 'test -f /jupyter/xxx'
!ansible {ugroup.name}_{unit_worker.name} -a 'test -f /exchange/xxx'
!ansible {ugroup.name} -m file -a 'path=/jupyter/xxx state=absent'
!ansible {ugroup.name} -m file -a 'path=/exchange/xxx state=absent'

Docker Swarmの設定を行う前に、各VCノードで Docker Engine が実行されていることを確認します。

In [ ]:
!ansible {ugroup.name} -a 'docker info'

manager用VCノードで Docker Swarmの初期セットアップを行います。

In [ ]:
manager_ip = gvars['vc_ipaddress']
!ansible {ugroup.name}_{unit_mgr.name} -a 'docker swarm init \
    --default-addr-pool={{{{docker_address_pool}}}} --advertise-addr={manager_ip}'

トークンの値を取得します。

In [ ]:
out = !ansible {ugroup.name}_{unit_mgr.name} -a 'docker swarm join-token -q worker'
if out[0].find('CHANGED') >= 0 or out[0].find('SUCCESS') >= 0:
    swarm_token = out[1]
    print(swarm_token)
else:
    raise

workerノードを追加します。

In [ ]:
!ansible {ugroup.name}_{unit_worker.name} -a \
    'docker swarm join --token {swarm_token} {manager_ip}:2377'

Docker Swarmのノード一覧を表示します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -a 'docker node ls'

Docker Swarmのノード数が起動したVCノードと一致していることを確認します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -m shell -a \
    'test $(docker node ls -q | wc -l) -eq {worker_nodes + 1}'

### ノード指定

In [ ]:

target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

### Moodle

In [ ]:
MOODLE_ROOT_DIR = 'moodle'

In [ ]:
!ansible {ugroup_name} -b -m file -a \
    'path=/home/vcp/{MOODLE_ROOT_DIR} state=directory owner={{{{ansible_user}}}}'

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=scripts/moodle.docker-compose.yml dest=/home/vcp/{MOODLE_ROOT_DIR}/docker-compose.yml backup=yes'

In [ ]:
# mcjとmoodleで共通のDockerネットワークを作成しておく（docker compose up 時に存在している必要あり）
!ansible {target_hub} -a 'docker network create --driver overlay --scope swarm --attachable mm_network'

In [ ]:
!ansible {target_hub} -a 'chdir=/home/vcp/{MOODLE_ROOT_DIR} \
    docker compose up -d'

In [ ]:
import time

max_retry = 30
for i in range(max_retry):
    cnt = !ansible {target_hub} -a "docker exec {MOODLE_ROOT_DIR}-moodle-1 grep -c '\$CFG->reverseproxy' /opt/bitnami/moodle/config.php"
    
    try:
        if int(cnt[1]) > 0:
            cnt = !ansible {target_hub} -a "docker exec {MOODLE_ROOT_DIR}-moodle-1 grep -c '{jupyterhub_fqdn}' /opt/bitnami/moodle/config.php"
            if int(cnt[1]) > 0:
                !ansible {target_hub} -a "docker exec {MOODLE_ROOT_DIR}-moodle-1 sed -i 's|{jupyterhub_fqdn}|{jupyterhub_fqdn}/moodle|' /opt/bitnami/moodle/config.php"
                break
            cnt = !ansible {target_hub} -a "docker exec {MOODLE_ROOT_DIR}-moodle-1 grep -c '{jupyterhub_fqdn}/moodle' /opt/bitnami/moodle/config.php"
            if int(cnt[1]) < 2:
                time.sleep(20)
                continue
    except Exception as e:
        print(e)
        time.sleep(20)
        continue
else:
    raise


### MCJ   

以下のセルをまとめ実行する。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

In [ ]:
!test -f group_vars/{ugroup_name}

In [ ]:
import random
import string
configproxy_auth_token = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
configproxy_auth_token

In [ ]:
!mkdir -p edit
!cp -n ./template/jupyterhub/jupyterhub/lms_web_service.py ./edit/
!cp -n ./template/jupyterhub/jupyterhub/jupyterhub_config.py ./edit/
!cp -n ./template/jupyterhub/jupyterhub/jupyterhub_params.yaml ./edit/

In [ ]:
# メールアドレスドメイン設定
email_domain = 'example.com'

# JupyterhubDB接続情報
db_user = 'jupyter'
db_password = 'PassWordDesu'

# Jupyterhub用LDAPサーバ（ローカルLDAP）
ldap_admin = 'Manager'
ldap_password = 'PassWordDesu'

# Jupyterhub 初期設定
jupyterhub_admin_users = ['admin']

# single-user notebook server コンテナイメージ
singleuser_image = 'mcj-cloudhub-nb:latest'

# 共有ディレクトリパス
home_directory_root = '/jupyter'
share_directory_root = '/exchange'

# dockerネットワーク
swarm_network = 'jupyterhub-network'

# ユーザを一意に識別するキー
lti_username_key = 'sub'

# ユーザのcookieの有効日数(0.25日=6時間)
cookie_max_age_days = 0.25

# Jupyterhubコンテナイメージ
jupyterhub_image = 'mcj-cloudhub:latest'

# single-user notebook server コンテナをDocker Swarmのどのノードで起動するか
# 「011-VCノード作成」で、worker_nodes(workerノード数)に0を指定した場合、"manager"を指定します。（"worker"を指定しても内部的には"manager"が指定されます）
# node_role = 'manager'
node_role = 'worker'


In [ ]:
import yaml
from pathlib import Path

path = Path('edit/jupyterhub_params.yaml')
with path.open() as f:
    params = yaml.safe_load(f)

params.update({
    'resource': {
        'groups': {
            'student': {
                'mem_limit': '1G',
                'cpu_limit': 0.5,
                'mem_guarantee': 0,
                'cpu_guarantee': 0,
            },
            'teacher': {
                'mem_limit': '1G',
                'cpu_limit': 1.0,
                'mem_guarantee': 0,
                'cpu_guarantee': 0,
            },
        },
    },
    'cull_server': {
        'cull_server_timeout': 600,
        'cull_server_every': 60,
        'cull_server_max_age': 0,
    },
})

with path.open(mode='w') as f:
    yaml.safe_dump(params, stream=f)

In [ ]:
# (例)
# jupyterhub_backend = '10.1.0.0/20'

jupyterhub_backend = '10.2.0.0/20'

In [ ]:
!ansible-playbook -v -e jupyterhub_backend={jupyterhub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

if gvars['worker_nodes'] == 0:
    node_role = 'manager'

lms_platform_id = ""
lms_cliend_id = ""

gvars.update({
    'swarm_network': swarm_network,
    'jupyterhub_backend': jupyterhub_backend,
    'jupyterhub_fqdn': jupyterhub_fqdn,
    'db_user': db_user,
    'db_password': db_password,
    'email_domain': email_domain,
    'jupyterhub_admin_users': jupyterhub_admin_users,
    'lms_platform_id': lms_platform_id,
    'lms_cliend_id': lms_cliend_id,
    'lms_api_token': lms_api_token if 'lms_api_token' in locals() else '',
    'get_course_member_method': get_course_member_method if 'get_course_member_method' in locals() else '',
    'singleuser_image': singleuser_image.split(':')[0],
    'singleuser_image_tag': singleuser_image.split(':')[1],
    'ldap_password': ldap_password,
    'ldap_admin': ldap_admin,
    'home_directory_root': home_directory_root,
    'share_directory_root': share_directory_root,
    'lti_username_key': lti_username_key,
    'cookie_max_age_days': cookie_max_age_days,
    'configproxy_auth_token': configproxy_auth_token,
    'jupyterhub_image': jupyterhub_image,
    'node_role': node_role,
    'ldap_admin': ldap_admin,
})

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}} state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/certs state=directory owner={{{{ansible_user}}}}'

In [ ]:
# 証明書配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=/notebooks/notebook/certs/fullchain.pem dest={{{{certs_dir}}}}'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=/notebooks/notebook/certs/privkey.pem dest={{{{certs_dir}}}}'

In [ ]:
try:
    !ansible {target_hub} -a \
        'openssl rsa -noout -text -in  {{{{certs_dir}}}}/privkey.pem'
except Exception as e:
    # 鍵の形式が異なる場合、正しく配置できていてもエラーになる場合があります。
    !ansible {target_hub} -a \
         'openssl ec -in {{{{certs_dir}}}}/privkey.pem -text -noout'

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/fullchain.pem'

In [ ]:
cert_owner = 33
cert_group = 33
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}} owner={cert_owner} \
    group={cert_group} state=directory'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/fullchain.pem \
    owner={cert_owner} group={cert_group}'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/privkey.pem \
    owner={cert_owner} group={cert_group} mode=0600'

In [ ]:
# 先にディレクトリ作成
!ansible {target_hub} -b -m file -a \
    'path={share_directory_root}/nbgrader state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={share_directory_root}/class state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/jupyterhub/ldap state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/jupyterhub/nginx state=directory owner={{{{ansible_user}}}}'

In [ ]:
# 共有ディレクトリに必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/jupytershare/nbgrader dest={share_directory_root}'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/jupytershare/class dest={share_directory_root}'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/directories/jupytershare/nbgrader/templates/teachers/nbgrader_config.py \
    dest={share_directory_root}/nbgrader/templates/teachers backup=yes'

# ホームディレクトリに必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/skelton dest={home_directory_root}'

# jupyterhubシステム構築に必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/jupyterhub dest={{{{base_dir}}}}/jupyterhub'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/nginx dest={{{{base_dir}}}}/jupyterhub'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/ldap dest={{{{base_dir}}}}/jupyterhub'

!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/docker-compose.yml backup=yes'

!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx/default.conf backup=yes'

!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/jupyterhub/sudoers dest={{{{share_directory_root}}}} rsync_opts=--chown=root:root,--chmod=600' --become

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-jupyterhub.yml -e jupyterhub_image={jupyterhub_image}

In [ ]:
!ansible {target_hub} -a \
    'docker images {{{{jupyterhub_image}}}}'

In [ ]:
!ansible {ugroup_name} -b -m file -a \
    'path={{{{notebook_dir}}}} state=directory owner={{{{ansible_user}}}}'

In [ ]:
!ansible {ugroup_name} -Dv -m synchronize \
    -a 'src=template/notebook dest={{{{base_dir}}}}'
!ansible {ugroup_name} -Dv -m template \
    -a 'src=template/notebook/image/ldap.conf dest={{{{base_dir}}}}/notebook/image/ backup=yes'
!ansible {ugroup_name} -Dv -m template \
    -a 'src=template/notebook/image/nbgrader_config.py dest={{{{base_dir}}}}/notebook/image/ backup=yes'

In [ ]:
!ansible-playbook -l {ugroup_name} playbooks/setup-jupyter-notebook.yml

In [ ]:
print(f'指定したイメージ名:{singleuser_image}')

!ansible {ugroup_name} -m shell -a 'docker images | \
    grep -e "{{{{singleuser_image}}}}"'

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=edit/lms_web_service.py dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=edit/jupyterhub_config.py dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=edit/jupyterhub_params.yaml dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'

In [ ]:
!ansible {target_hub} -a 'chdir={{{{base_dir}}}}/jupyterhub \
    docker stack deploy -c docker-compose.yml {{{{ugroup_name}}}}'

### JHのヘルスチェック

ACL設定等を行っていなければ通信できないため、エラーになる。

In [ ]:
import time

# 規定回数
retry_max = 18
err = None

for retry in range(retry_max):
    try:
        !ansible {target_hub} -m uri -a "url=https://{master_fqdn}/hub/health"
        break

    except Exception as e:
        print("retry")
        err = e
        time.sleep(10)
else:
    raise err

## 外部ツール設定  
Moodleにアクセスし、外部ツール設定を行う。  
設定後、外部ツール情報を確認し、以下のセルでパラメータ指定をおこなう。

### パラメータ表示  
以下のパラメータを外部ツール設定画面で指定する。  
**アクセスすると、`Reverse proxy enabled so the server cannot be accessed directly.`というエラーが出る場合がある。  
この場合、Moodleの設定（`config.php`）に失敗している。`config.php`の設定を行っているセルを再度実行する。**（Moodleの稼働に少し遅れてdocker-compose.ymlで設定した環境変数をもとにした設定が反映されるため）

In [ ]:
print('MoodleのURL:', f'https://{jupyterhub_fqdn}/moodle')
print(f'ログイン情報: id=admin password={moodle_admin_password}')
print('Tool URL:', f'https://{jupyterhub_fqdn}/')
print('Initiate login URL:', f'https://{jupyterhub_fqdn}/hub/lti13/oauth_login')
print('Redirection URI(s):', f'https://{jupyterhub_fqdn}/hub/lti13/oauth_callback')

if not 'get_course_member_method' in globals() or not get_course_member_method == 'moodle_api':
    res = !ansible {target_hub} -a \
        'cat  {{{{jupyterhub_dir}}}}/jupyterhub/public_key_nrps.pem'
    print('Public key:')
    print('\n'.join(res[1:]))

### パラメータ指定  
Moodleで設定した外部ツール情報を確認し、以下の項目を設定する。

In [ ]:
# lti1.3認証連携情報(moodle等)
# lms プラットフォームID (例) 'www.sample.org'
lms_platform_id = 

# lms クライアントID 
lms_cliend_id = 

### 設定  
以下まとめ実行する

In [ ]:
import os
%run scripts/group.py
update_group_vars(
    ugroup_name,
    lms_platform_id=lms_platform_id,
    lms_cliend_id=lms_cliend_id,
)

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'

In [ ]:
!ansible {target_hub} -a 'docker service update --env-add LMS_PLATFORM_ID={lms_platform_id} {ugroup_name}_jupyterhub'
!ansible {target_hub} -a 'docker service update --env-add LMS_CLIENT_ID={lms_cliend_id} {ugroup_name}_jupyterhub'